# Report 1

The goal of the first part of the class is to 
 - Be able to implement a finite element solver for linear elasticity.
 - Be able to perform a convergence analysis.
 - Investigate locking issue and be able to develop a mixed formulation for curing locking in the case of almost incompressible materials.
    
Your report should summarize and present synthetically your work on these items. We give below a possible guide to write the report. Personalized analyses and comments are particularly welcome. Your are not obliged to follow the following format step and by step. But you should include in your report the main steps. 

Some suggestions:
 - Write concisely and effectively.
 - Comment your results.
 - The quality of the figures is important.
 - Report only the minimal number of figures (of excellent quality) to effectively communicate your results.
 - You can write in English of French.
 - Use Latex or Jupyter notebooks for writing your report. But if you use Jupyter notebook, the format should be professional and you should produce a pdf as output. 
 

# Linear elasticity: convergence analysis and mixed formulation for almost incompressible materials

Let us consider a linear elastic cylinder with an internal radius $R_{\mathrm{in}}=R-t/2$ and external radius  $R_{\mathrm{out}}=R+t/2$, where $R$ is the average radius and $t$ the thickness. The cylinder is sumbitted to an internal pressure $p_{\mathrm{in}}$ and an external pressure $p_{\mathrm{out}}$. We will denote by $E$ and $\nu$ its Young modulus and Poisson ratio. We consider a 2D model for the cylinder modelling only its cross-section under plane-strain assumption.

# I. Preliminaries (work on this at home, not during the TP sessions)

1. Give a variational formulation of the problem.
2. Write the problem formulation in a dimensionless form. Using the dimensional analysis and the linearity of the problem, show that the solution depends non-trivially only on two independent parameters: the aspect ratio $t/R$ and the Poisson ratio $\nu$, and that without loss of generality one can set $E=1$, $R=1$, and $p_{\mathrm{out}}=0$, $p_{\mathrm{in}}=1.0$. 
3. Obtain the analytical solution of the problem.
4. Obtain an approximate analytical solution with a simplified method in the case $t/R\ll 1$ (`formule du chaudronnier`). Discuss the limit of validity of this solution.


# II. Finite element solver and convergence analysis

1. Write a finite element solver for the problem. Comment the different steps (see the FEniCS documented demo as an example). 
2. Encapsulate the solver in a function (as in TP.1.2.2)
3. For the case $\nu=0.3$, $t/R=0.5$, perform a convergence analysis, following as example the notebook `1.4-Error-Convergence.ipynb`. Check if you obtain the convergence rates predicted by apriori estimates in  `L2` and `H1` norm for `P1` and `P2` elements. Summarize the result in a loglog plot and compare to the convergence rate predicted by the theory.
5. Comment the results and draw conclusions.



# III. Mixed formulation in the almost incompressible case

1. (To do at home) Show that incompressible materials are caracterized by a Poisson ratio $\nu=0.5$. Recall, showing why, which are the minimum and maximum values of the Poisson ratio for a linear elastic material.


The numerical solution of the equation of linear elasticity for almost incompressible materials (e.g. $\nu=0.4999999$) presents particular difficulties. When using a finite element discretisation there is a numerical locking problem *(verrouillage numérique)* that strongly affects the convergence rate. In the following we try to put into evidence this problem and propose a solution based on a mixed variational formulation, introducing a splitting of the stress tensor into deviatoric and volumetric parts and the use of appropiate discretisations.

## Numerical experiments

 - Repeat the numerical convergence analysis for the cylinder for $\nu=0.48$ et $\nu=0.4999$. 
 - Plot the fields representing the deviatoric and the isotropic part of the  stress  tensor for the two values of the Poisson ratio. 
 - Plot the error norm of the solution as a function of the mesh size in logarithmic scale. 
 - Comment the results. What can be deduced on the convergence rate for almost incompressible materials? Is this result compatible with the mathematical result on the convergence rate of finite element approximation. 

## Mixed variational formulation $(u,p)$

1. Show that the elastic energy density of a linear elastic material can be decomposed into a volumetric and deviatoric part, as follows:
$$
W(u) = \dfrac{1}{2}k \,\mathrm{div} u^2 +\mu \epsilon_d(u)\cdot  \epsilon_d(u)
$$
where $\epsilon_d(u)=\epsilon(u)-\epsilon_I(u)$ is the deviatoric part of the strain tensor and $\epsilon_I(u)=\frac{\mathrm{div}{u}}{3} I$ its isotropic part. Give the expression of the bulk modulus $k$ as a function of $(E,\nu)$ or $(\lambda,\mu)$ in 2D and 3D. 

2. Show that $W$  can be rewritten in the following form (that is a Legendre tranformation, see *e.g.* https://en.wikipedia.org/wiki/Legendre_transformation)
\begin{equation}
W(u)=\max_p W(u,p), \qquad W(u,p)  = p\,\mathrm{div} u-\dfrac{p^2}{2\,k} +\mu \epsilon_d(u)\cdot  \epsilon_d(u)
\end{equation}

3. Starting from the principle of minimum of potential energy, show that the linear elasticity can be formulated through the following *mixed variational principle*:
$$
\min_{u\in \mathcal{V}}\, \max_{p\in \mathcal{P}} \int_\Omega W(u,p)\,dx -\ell(u)
$$
$\ell(u)$ being the linear functional representing the potential of external forces. Here $\mathcal{V}=H^1(\Omega)\times H^1(\Omega)$ (+ boundary conditions on $u$) and $\mathcal{P}= L^2(\Omega)$ are the spaces of admissible displacement and pressure fields, respectively.
Remark that in the perfectly incompressible case ($k\to\infty$) the mixed formulation above coincides with the principle of the minimum of potential energy, where the incompressibility constraint  $\mathrm{div} u =0$ is imposed through the Lagrange multiplier method ($p$ being the multiplier). 

4. Show that the stationarity conditions of the mixed formulation above are in the following form:
\begin{eqnarray}
a(u,v)+b(v,p)&=&l(v)\qquad \forall v\in\mathcal V\\
b(u,q)+ c(p,q)&=&0,\qquad \forall q\in\mathcal P
\end{eqnarray}
Give the explicit expressions of the bilinear forms $a$, $b$, $c$. Note that the second equation is a weak form of the volumetric part of the constitutive relation.

5.  Write a numerical code to solve the mixed problem above for the case of the cylinder. You can find details on how to implement mixed formulation here: https://fenicsproject.org/documentation/dolfin/2016.1.0/python/demo/documented/stokes-taylor-hood/python/documentation.html

6. Compare the results obtained with the following discretisations for the displacement and pressure subspaces:
 * CG1-CG1 (P1 for displacement and pression)
 * CG2-CG1 (P2 for displacement and P1 for pressure)
 * CG1-DG0 (P1 for displacement and piecewise constant for the pressure)
 
For the different cases above show:
 * The convergence diagram for the error norm.
 * The pressure field on a relatively coarse mesh.

# Optional: Cook's membrane
Perform the convergence analysis on the following problem: the Cook's membrane. In this classical test case, we set $E = 250, ν = 0.4999, V = 100.$
<img src="files/cook.png" alt="Drawing" style="width: 300px;"/>

# References
[1] H.P.Langtangen, A.Logg, Solving PDEs in Minutes - The FEniCS Tutorial Volume I (preprint) 2016 https://github.com/alogg/fenics-tutorial/raw/master/pub/pdf/fenics-tutorial1-4print.pdf

[2] B.Szabó, I.Babuška, Introduction to Finite Element Analysis: Formulation, Verification and Validation, Wiley 2011, ISBN: 978-0-470-97728-6